# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hamilton,39.1834,-84.5333,303.50,60,0,2.57,US,1721147410
1,1,punta arenas,-53.1500,-70.9167,279.21,75,40,4.12,CL,1721147909
2,2,nova olimpia,-14.7972,-57.2881,304.34,27,0,2.00,BR,1721147911
3,3,palikir - national government center,6.9248,158.1611,298.56,94,100,2.06,FM,1721147912
4,4,kuraymah,18.5500,31.8500,312.13,11,100,1.79,SD,1721147913


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    fill_alpha = 0.5,
    title = "Cities in City_data_df",
    xlabel = "Latitude",
    ylabel = "Longitude")

# Display the map
map1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# Filter the city_data_df DataFrame for my family's ideal weather for vacationing
ideal_weather_df = city_data_df.loc[
    (city_data_df["Humidity"] <= 70) &
    (city_data_df["Max Temp"] >= 273) &
    (city_data_df["Max Temp"] <= 297) &
    (city_data_df["Wind Speed"] <= 10) &
    (city_data_df["Cloudiness"] <= 50) 
    ]

# Drop any rows with null values
ideal_weather_df.dropna().reset_index(drop=True)

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,greytown,-29.0642,30.5928,282.81,48,0,0.71,ZA,1721147932
25,25,ushuaia,-54.8000,-68.3000,280.94,70,20,1.03,AR,1721147936
62,62,melfort,52.8667,-104.6177,291.22,52,2,2.57,CA,1721147978
71,71,bodo,67.2800,14.4050,295.34,53,20,2.57,NO,1721147988
84,84,ust-nera,64.5667,143.2000,288.45,47,6,1.96,RU,1721148005
...,...,...,...,...,...,...,...,...,...,...
549,549,banket,-17.3833,30.4000,291.50,25,3,1.73,ZW,1721148555
551,551,el galpon,-25.3807,-64.6526,292.83,23,1,3.06,AR,1721148557
566,566,pico truncado,-46.7949,-67.9573,280.04,63,26,6.35,AR,1721148575
567,567,syktyvkar,61.6764,50.8099,296.00,64,0,2.00,RU,1721148576


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
21,greytown,ZA,-29.0642,30.5928,48,
25,ushuaia,AR,-54.8000,-68.3000,70,
62,melfort,CA,52.8667,-104.6177,52,
71,bodo,NO,67.2800,14.4050,53,
84,ust-nera,RU,64.5667,143.2000,47,
...,...,...,...,...,...,...
549,banket,ZW,-17.3833,30.4000,25,
551,el galpon,AR,-25.3807,-64.6526,23,
566,pico truncado,AR,-46.7949,-67.9573,63,
567,syktyvkar,RU,61.6764,50.8099,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address_json = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
greytown - nearest hotel: Greytown hotel
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
melfort - nearest hotel: No hotel found
bodo - nearest hotel: Zefyr hotell
ust-nera - nearest hotel: гостиница "Солнечная"
dudinka - nearest hotel: Виктория
puerto natales - nearest hotel: Dorotea Patagonia Hostel
isafjordur - nearest hotel: Hótel Horn
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
altamont - nearest hotel: Quality Inn
khorugh - nearest hotel: кивекас
urangan - nearest hotel: Ramada
guerrero negro - nearest hotel: Plaza sal paraiso
azangaro - nearest hotel: Hotel Princess Jairito
port alfred - nearest hotel: No hotel found
nueva helvecia - nearest hotel: Hotel del Prado
castillos - nearest hotel: A mi gente
thompson - nearest hotel: Thompson Inn
sandpoint - nearest hotel: K2 Inn
la rioja - nearest hotel: No hotel found
peniche - nearest hotel: Residencial Maciel
talnakh - nearest hotel: Талнах
minas de marcona - nearest hotel: No hotel foun

,City,Country,Lat,Lng,Humidity,Hotel Name
21,greytown,ZA,-29.0642,30.5928,48,Greytown hotel
25,ushuaia,AR,-54.8000,-68.3000,70,Apart Hotel Aires del Beagle
62,melfort,CA,52.8667,-104.6177,52,No hotel found
71,bodo,NO,67.2800,14.4050,53,Zefyr hotell
84,ust-nera,RU,64.5667,143.2000,47,"гостиница ""Солнечная"""
...,...,...,...,...,...,...
549,banket,ZW,-17.3833,30.4000,25,No hotel found
551,el galpon,AR,-25.3807,-64.6526,23,No hotel found
566,pico truncado,AR,-46.7949,-67.9573,63,No hotel found
567,syktyvkar,RU,61.6764,50.8099,64,Гостиница Строитель


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    fill_alpha = 0.5,
    title = "My Family's Ideal Cities to Vacation",
    xlabel = "Latitude",
    ylabel = "Longitude",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)